In [1]:
import pandas as pd
import numpy as np
import re
import textract

In [2]:
import tabula

In [3]:
#extraigo texto
archivo= 'tresalia\Edo Cta\BNY PDF Inveralia Inc.pdf'

In [4]:
text = textract.process(archivo)

In [5]:
#cambio formato
encoding = 'utf-8'
text_str=text.decode(encoding)

In [6]:
#separo
pdf=re.split(r"[~\r\n]+", text_str)

In [7]:
information_pdf=pd.DataFrame(pdf).rename(columns={0:"informacion"})
information_pdf.head()

,informacion
0,Statement from 04/01/20 through 04/30/20
1,XXXXXXXXXX % LOEB BLOCK & PARTNERS LLP 505 PAR...
2,0
3,99999999
4,Page 1 of 2 BZ 9999999999 ( 0 )


In [8]:
pdf

['Statement from 04/01/20 through 04/30/20',
 'XXXXXXXXXX % LOEB BLOCK & PARTNERS LLP 505 PARK AVE 8TH FL NEW YORK NY 10022-1106',
 '0',
 '99999999',
 'Page 1 of 2 BZ 9999999999 ( 0 )',
 'For personal assistance, call XXXXXXXXXXX BNY Mellon Wealth Management 200 Park Avenue New York, NY 10166 999-999-9999',
 'After business hours, call our Digital Online Support Center at 999-999-9999',
 'REMINDER: SELF-SERVICE BANKING AND 24/7 ACCOUNT ACCESS IS AVAILABLE THROUGH ONLINE AND MOBILE BANKING, WHERE YOU CAN CHECK BALANCES, TRANSFER FUNDS, VIEW TRANSACTIONS, LOCATE ATMS AND MAKE MOBILE DEPOSITS. IF YOU ARE NOT ENROLLED IN ONLINE OR MOBILE BANKING OR NEED ASSISTANCE, PLEASE CONTACT OUR WEALTH ONLINE SUPPORT TEAM AT 800-880-5631(WEEKDAYS, 6 A.M.- 10 P.M. AND WEEKENDS, 8 A.M.- 8 P.M. ET).',
 'Business Checking 9999999999',
 'Account Summary',
 'BEGINNING BALANCE',
 'DEPOSITS WIRE TRANSFER CREDITS OTHER CREDITS TOTAL CREDITS',
 'CHECKS PAID WIRE TRANSFER DEBITS SERVICE CHARGE OTHER DEBITS TOTAL

In [9]:
#periodo
periodo_lista = []
for i in pdf:
    if re.search('from',i) and re.search('through',i):
        periodo_lista.append(i)
periodo=periodo_lista[0]
periodo

'Statement from 04/01/20 through 04/30/20'

In [10]:
#nombre
cuenta_lista = []
for i in pdf:
    if re.search('Business Checking',i):
        cuenta_lista.append(i)
cuenta= cuenta_lista[0]
cuenta

'Business Checking 9999999999'

In [11]:
datos_lista = []
for i in pdf:
    if re.search('%',i):
        datos_lista.append(i)
nombre=datos_lista[0].split('%')[0]
direccion=datos_lista[0].split('%')[1]

In [12]:
datos=dict(zip(['nombre','direccion'],[nombre,direccion]))
datos

{'nombre': 'XXXXXXXXXX ',
 'direccion': ' LOEB BLOCK & PARTNERS LLP 505 PARK AVE 8TH FL NEW YORK NY 10022-1106'}

In [13]:
indices_cuenta=[]
for i in range(len(pdf)):
    if re.search('account activity', pdf[i].lower()):
        indices_cuenta.append(i)

In [33]:
indices_service_charge=[]
for i in range(len(pdf)):
    if re.search('service charge detail', pdf[i].lower()):
        indices_service_charge.append(i)

In [34]:
indices_service_charge

[58]

In [53]:
tabla=pdf[indices_cuenta[0]:indices_service_charge[0]+3]

In [54]:
tabla

['Account Activity',
 'Date Posted',
 'Transaction Description',
 'Checks/ Other Debits',
 '03-31 ENDING BALANCE PREVIOUS STATEMENT ..................................................................',
 '04-14',
 'WIRE TRANSFER CREDIT REF # 000000000009999',
 'O/B:XXXXXXXX WIRE TRANSFER CREDIT REF # 000000000009999',
 'O/B:XXXXXX',
 '........................................................ ........................................................',
 'Deposits/ Other Credits',
 '2,809.65',
 '1,700.00',
 '1,200.00',
 '\x0cStatement from 04/01/20 through 04/30/20',
 '9999999',
 'Page 2 of 2 Account #: 6310019982',
 'Date Posted',
 'Transaction Description',
 'Checks/ Other Debits',
 'WIRE TRANSFER CREDIT REF # 000000000203008 ........................................................ O/XXXXXXXX',
 'Deposits/ Other Credits',
 '52,000.00',
 '04-15',
 'WIRE TRANSFER-OUT BEN:XXXXX',
 '8,286.00',
 '04-17',
 'WIRE TRANSFER-OUT',
 'BEN:XXXXXXXXXXXXXXXX XXX',
 'WIRE TRANSFER FEE',
 '6,431.00 30.00',


In [56]:
#fechas
lista_fechas=[]
for i in tabla:
    if re.search(r'\d{2}-\d{2}', i):
        lista_fechas.append(i)

In [57]:
lista_fechas

['03-31 ENDING BALANCE PREVIOUS STATEMENT ..................................................................',
 '04-14',
 '04-15',
 '04-17',
 '04-30']

In [58]:
lista_fechas_2=[]
for i in lista_fechas:
    j=re.findall(r'\d{2}-\d{2}', i)
    lista_fechas_2.append(j)

In [59]:
lista_fechas_2

[['03-31'], ['04-14'], ['04-15'], ['04-17'], ['04-30']]

In [60]:
fechas=[]
for sublist in lista_fechas_2:
    for item in sublist:
        if item.startswith('00')==False:
            fechas.append(item)

In [61]:
fechas

['03-31', '04-14', '04-15', '04-17', '04-30']

In [62]:
#transaction description
lista_descripcion=[]
for i in tabla:
    if re.search(r'WIRE|ENDING|SERVICE CHARGE',i):
        lista_descripcion.append(i)

In [63]:
lista_descripcion

['03-31 ENDING BALANCE PREVIOUS STATEMENT ..................................................................',
 'WIRE TRANSFER CREDIT REF # 000000000009999',
 'O/B:XXXXXXXX WIRE TRANSFER CREDIT REF # 000000000009999',
 'WIRE TRANSFER CREDIT REF # 000000000203008 ........................................................ O/XXXXXXXX',
 'WIRE TRANSFER-OUT BEN:XXXXX',
 'WIRE TRANSFER-OUT',
 'WIRE TRANSFER FEE',
 'SERVICE CHARGE MONTHLY MAINT FEE']

In [69]:
#depositos/créditos tablas

cheques_depositos=[]
for i in tabla:
    if re.search(r'\.\d{2}$',i):
        cheques_depositos.append(i)

In [73]:
cheques_depositos_2=[]
for i in cheques_depositos:
    j=i.split(' ')
    cheques_depositos_2.append(j)

In [76]:
#aplano
cheques_depositos_3=[]
for sublist in cheques_depositos_2:
    for item in sublist:
            cheques_depositos_3.append(item)

In [77]:
cheques_depositos_3

['2,809.65',
 '1,700.00',
 '1,200.00',
 '52,000.00',
 '8,286.00',
 '6,431.00',
 '30.00',
 '15.00']

In [79]:
#arrego Fechla
contador=0
fecha_2=[]
for i in lista_descripcion:
    if re.search('WIRE TRANSFER CREDIT',i):
        print(i)

WIRE TRANSFER CREDIT REF # 000000000009999
O/B:XXXXXXXX WIRE TRANSFER CREDIT REF # 000000000009999
WIRE TRANSFER CREDIT REF # 000000000203008 ........................................................ O/XXXXXXXX


In [85]:
#no se como arreglar fechas de forma automatica
fechas.insert(2,'0-14')
fechas.insert(2,'0-14')

In [88]:
fechas.insert(6,'04-17')

In [89]:
estado=pd.DataFrame(list(zip(fechas,lista_descripcion,cheques_depositos_3)), columns=['fecha','descripción', 'cheques,depósito'])

In [90]:
#estado se puede exportar
estado

,fecha,descripción,"cheques,depósito"
0,03-31,03-31 ENDING BALANCE PREVIOUS STATEMENT .........,"2,809.65"
1,04-14,WIRE TRANSFER CREDIT REF # 000000000009999,"1,700.00"
2,0-14,O/B:XXXXXXXX WIRE TRANSFER CREDIT REF # 000000...,"1,200.00"
3,0-14,WIRE TRANSFER CREDIT REF # 000000000203008 ......,"52,000.00"
4,04-15,WIRE TRANSFER-OUT BEN:XXXXX,"8,286.00"
5,04-17,WIRE TRANSFER-OUT,"6,431.00"
6,04-17,WIRE TRANSFER FEE,30.00
7,04-30,SERVICE CHARGE MONTHLY MAINT FEE,15.00
